In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from main import TradingBot
from quiz import Quiz

tb = TradingBot()
quiz = Quiz(tb)

In [2]:
tb.loadData(timePeriod='2y', label='NIFTY 100', forceUpdate=True)
tb.computeSignal()

100%|██████████| 102/102 [01:43<00:00,  1.02s/it]


In [ ]:
quiz.random(5)